In [11]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import miceforest as mf
import re
import openpyxl

In [4]:
data = pd.read_csv('./csv_data/INS-W_1/FeatureData/sleep.csv')
data.columns = data.columns.str.replace(":","_",regex=True)
# df.drop(columns=['Unnamed: 0'], inplace=True)
data.set_index(['pid','date'])
data.head(10)

,Unnamed_ 0,pid,date,f_slp_fitbit_sleep_summary_rapids_sumdurationafterwakeupmain_14dhist,f_slp_fitbit_sleep_summary_rapids_sumdurationasleepmain_14dhist,f_slp_fitbit_sleep_summary_rapids_sumdurationawakemain_14dhist,f_slp_fitbit_sleep_summary_rapids_sumdurationtofallasleepmain_14dhist,f_slp_fitbit_sleep_summary_rapids_sumdurationinbedmain_14dhist,f_slp_fitbit_sleep_summary_rapids_avgefficiencymain_14dhist,f_slp_fitbit_sleep_summary_rapids_avgdurationafterwakeupmain_14dhist,...,f_slp_fitbit_sleep_intraday_rapids_stddurationasleepunifiedmain_norm_weekend,f_slp_fitbit_sleep_intraday_rapids_stddurationawakeunifiedmain_norm_weekend,f_slp_fitbit_sleep_intraday_rapids_mindurationasleepunifiedmain_norm_weekend,f_slp_fitbit_sleep_intraday_rapids_mindurationawakeunifiedmain_norm_weekend,f_slp_fitbit_sleep_intraday_rapids_mediandurationasleepunifiedmain_norm_weekend,f_slp_fitbit_sleep_intraday_rapids_mediandurationawakeunifiedmain_norm_weekend,f_slp_fitbit_sleep_intraday_rapids_ratiocountasleepunifiedwithinmain_norm_weekend,f_slp_fitbit_sleep_intraday_rapids_ratiocountawakeunifiedwithinmain_norm_weekend,f_slp_fitbit_sleep_intraday_rapids_ratiodurationasleepunifiedwithinmain_norm_weekend,f_slp_fitbit_sleep_intraday_rapids_ratiodurationawakeunifiedwithinmain_norm_weekend
0,0,INS-W_001,2018-04-03,0.0,347.0,17.0,0.0,364.0,95.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,INS-W_001,2018-04-04,0.0,742.0,56.0,0.0,798.0,93.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,INS-W_001,2018-04-05,5.0,1313.0,85.0,0.0,1403.0,93.666667,1.666667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,INS-W_001,2018-04-06,5.0,1667.0,100.0,0.0,1772.0,94.250000,1.250000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,INS-W_001,2018-04-07,5.0,2264.0,133.0,0.0,2402.0,94.400000,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,INS-W_001,2018-04-08,5.0,2709.0,162.0,0.0,2876.0,94.333333,0.833333,...,0.0,-0.159151,0.0,0.0,-0.104167,0.0,-0.415435,0.415435,0.003823,-0.003823
6,6,INS-W_001,2018-04-09,5.0,3326.0,170.0,0.0,3501.0,95.000000,0.714286,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,INS-W_001,2018-04-10,5.0,3736.0,183.0,0.0,3924.0,95.250000,0.625000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,INS-W_001,2018-04-11,5.0,4245.0,217.0,0.0,4467.0,95.111111,0.555556,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,INS-W_001,2018-04-12,5.0,4839.0,275.0,0.0,5129.0,94.700000,0.500000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Normalize the data
scaler = MinMaxScaler()
data_numeric = data.select_dtypes(include=[float])
data_normalized = pd.DataFrame(scaler.fit_transform(data_numeric), columns=data_numeric.columns)
data_normalized = data_normalized.dropna(axis=1, how='all')

true_values_ix = data_normalized.dropna(subset=['f_slp_fitbit_sleep_intraday_rapids_stddurationasleepunifiedmain_norm_weekend']).sample(frac=0.1).index
true_values = data_normalized['f_slp_fitbit_sleep_intraday_rapids_stddurationasleepunifiedmain_norm_weekend']
data_normalized.loc[true_values_ix ,'f_slp_fitbit_sleep_intraday_rapids_stddurationasleepunifiedmain_norm_weekend'] = np.nan
# df_norm_np = data_normalized.to_numpy()

c:\Users\darkenral\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
c:\Users\darkenral\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))


In [6]:
# Create kernel. 
kds = mf.ImputationKernel(
  data_normalized,
  num_datasets=1,
  random_state=1
)

# Run the MICE algorithm for 2 iterations
kds.mice(
    iterations=1
)

# Return the completed dataset.
df_imputed = kds.complete_data()

In [13]:
df_imputed.to_excel('./imputed_data/INS-W_1/FeatureData/sleep_imputed.xlsx')

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error as mape


158505177293176.7